In [1]:
class ItemBasedCollaborativeFilteringRecommender(object):
    
    def fit(self, URM_csr):
        self.IRM = sparse.csr_matrix(URM_csr.transpose())
        self.similarities = cosine_similarity(self.IRM, dense_output=False)
        #self.similarities.data = np.where(self.similarities.data >= 0.05, self.similarities.data, 0)
        self.URM_csr = URM_csr
    
    def recommend(self, user_id, at=10, remove_seen=True):
        user = np.array(self.URM_csr.getrow(user_id).todense())[0]
        itemPopularity = self.similarities.dot(user)
        self.popularItems = np.argsort(itemPopularity)
        self.popularItems = np.flip(self.popularItems, axis = 0)
        
        if remove_seen:
            unseen_items_mask = np.in1d(self.popularItems, self.URM_csr[user_id].indices,
                                        assume_unique=True, invert = True)

            unseen_items = self.popularItems[unseen_items_mask]
            
            recommended_items = unseen_items[0:at]

        else:
            recommended_items = self.popularItems[0:at]
            
        #recommended_items = " ".join(str(i) for i in recommended_items)
        return recommended_items